# micrograd

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5)
ys = f(xs)
plt.plot(xs, ys)

The derivative of a function ***f(x)*** is the amount ***f(x)*** changes when a small value ***h*** is added to ***x***, as ***h*** approaches `0`.
$$ \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

In [ ]:
h = 0.000001
x = 2/3
(f(x+h) - f(x)) / h # Dividing by h 'normalizes' the difference

In [ ]:
# More complex function d(a, b, c) = a * b + c
h = 0.0001

# Inputs
a = 2.0
b =  -3.0
c = 10.0

d1 = a * b + c
a += h
d2 = a * b + c

print("d1:", d1)
print("d2:", d2)
print("Slope:", (d2-d1)/h)
# Note: da/dd = b = -3.0
# Note: db/dd = a = 2.0
# Note: dc/dd = so the slope will be 1

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op="", label="") -> None:
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self) -> str:
        return f"Value(data={self.data})"

    def __neg__(self):
        return self * -1
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _children=(self, other), _op="+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other
    
    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _children=(self, other), _op="*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __rmul__(self, other): # other, self: Reverses the args of __mul__ if it fails
        return self * other

    def __truediv__(self, other):
        return self * (other**-1)
    
    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def tanh(self):
        x = self.data
        tanh_x = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(data=tanh_x, _children=(self,), _op='tanh')

        def _backward():
            local_grad = 1 - tanh_x**2
            self.grad += local_grad * out.grad
        out._backward = _backward

        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int/float powers"
        out = Value(data=self.data**other, _children=(self, ), _op=f"^{other}")

        # d/dx(x^n) = n*x^n-1
        def _backward():
            n = other
            x = self.data
            self.grad += (n * (x**(n-1))) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out =  Value(data=math.exp(x), _children=(self, ), _op='exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def backward(self):
        # Topological sort
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v) # Append a node only after it's children have been appended
                return topo
        build_topo(self)
            
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()




a = Value(2.0, label="a")
b = Value(-3.0, label="b")
c = Value(10.0, label="c")
e = a * b; e.label = 'e'   # Intermediate variable for better visualization
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f

L

In [ ]:
from graphviz import Digraph

def trace(root):
    """Builds a set of all nodes and edges in the graph"""
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"}) # LR means Left-to-Right
    
    nodes, edges = trace(root)
    for node in nodes:
        uid = str(id(node))
        # For any value in the graph create a rectangular ('record') node for it
        dot.node(name=uid, label=f"{{{node.label} | data: {node.data:.4f} | grad: {node.grad:.4f} }}", shape="record")
        if node._op:
            # If this value is the result of an operation, create an op node for it
            dot.node(name = uid + node._op, label = node._op)
            # and connect this node to it.
            dot.edge(uid + node._op, uid)
    
    for parent, child in edges:
        # Connect the parent to the op node of it's child
        dot.edge(str(id(parent)), str(id(child)) + child._op)
    return dot

## Simple Neuron

![Simple Neuron]("static/simple_neuron.jpeg")

In [ ]:
# Inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# Weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# Bias of the neuron
b = Value(6.8813735870195432); b.label = 'b'

# x1w1 + x2w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'

x2w2 = x2 * w2
x2w2.label = 'x2*w2'

x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b
n.label = 'n'

# Activation function
e = (2*n).exp(); e.label = 'e'
o = (e - 1) / (e + 1)
o.label = 'o'
# o = n.tanh()


In [ ]:
draw_dot(o)

In [ ]:
o.backward()


## Neuron

![Neural Net]("./static/neural_net.jpeg")

In [34]:
class Neuron:
    def __init__(self, number_of_inputs) -> None:
        self.weights = [Value(random.uniform(-1, 1)) for _ in range(number_of_inputs)]
        self.bias = Value(random.uniform(-1, 1))
    
    def __call__(self, xs):
        """Takes the input for a neuron and gives x * w + b."""
        activation = sum(wi*xi for wi, xi in zip(self.weights, xs)) + self.bias
        out = activation.tanh()
        return out

    def parameters(self):
        return self.weights + [self.bias]

class Layer:
    """List of neurons that all get the same inputs."""
    def __init__(self, number_of_inputs, number_of_output) -> None:
        """Every neuron has number_of_inputs inputs and there are number_of_outputs neurons"""
        self.neurons = [Neuron(number_of_inputs) for _ in range(number_of_output)]
    
    def __call__(self, xs):
        outs = [neuron(xs) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [parameter for neuron in self.neurons for parameter in neuron.parameters()]

class MLP():
    """Multi-Layer Perceptron"""

    def __init__(self, number_of_inputs, number_of_outputs) -> None:
        layer_sizes = [number_of_inputs] + number_of_outputs
        self.layers = [Layer(layer_sizes[i], layer_sizes[i+1]) for i in range(len(number_of_outputs))]
    
    def __call__(self, xs):
        for layer in self.layers:
            # Propagate the inputs through the layers
            xs = layer(xs)
        return xs

    def parameters(self):
        return [parameter for layer in self.layers for parameter in layer.parameters()]

x = [2.0, 3.0]
# n = Layer(2, 3) # Neuron with two inputs
# n(x)
net = MLP(3, [4, 4, 1])
net([1.0, 2.0, 3.0])

Value(data=-0.5299456467341261)

In [38]:
data = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

target_predictions = [1.0, -1., -1., 1.]
learning_rate = 0.1

# Training Loop
for iteration in range(50):
    ## Forward Pass
    predictions = [net(row) for row in data]
    loss = sum([(prediction - target)**2 for target, prediction in zip(target_predictions, predictions)])

    ## Backward Pass
    ### Clear gradients
    for param in net.parameters():
        param.grad = 0.0
    loss.backward()

    ## Update
    for param in net.parameters():
        # We subtract because we want to reduce the value of the loss.
        param.data -= param.grad * learning_rate
    
    print(iteration, loss.data)

0 0.017873985476960234
1 0.0166423501269899
2 0.01560193391864436
3 0.014709807198896228
4 0.01393503022695108
5 0.013254743970216636
6 0.012651696688067173
7 0.012112626172873384
8 0.011627171022239405
9 0.011187119538893033
10 0.010785880146583604
11 0.01041810072202407
12 0.010079390221759427
13 0.009766111947040388
14 0.009475227854930206
15 0.009204179817521491
16 0.008950798008479234
17 0.008713229467139271
18 0.00848988185080228
19 0.008279378745666864
